# Cross-subject decoding Logic-Sensory rule gating (SR)
### Ex: Gating red rule information ([both, red] v. [not both, red] v. [either, red] v. [neither, red])

#### Takuya Ito
#### 2/28/2018

In [75]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group
import nibabel as nib
tools_group = reload(tools_group)

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [76]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 1.0 Run across subject decoding on RED SR rules
### 4-way decoding
* both red
* notboth red
* either red
* neither red

## 1.1 Load data for RED SR integration

In [77]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srRed'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools_group.loadInputActivity(subj,sr_rule)
    scount += 1

/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


#### Run decoding 

In [80]:
nproc = 30
ncvs = 1
rois = np.arange(nParcels)

distances_baseline_srRed = tools_group.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)

#### Compute statistics

In [97]:
sr_rule = 'srRed'
statistics_srRed = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_srRed.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_srRed[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_srRed[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_srRed[roicount,0] = np.mean(distances_baseline_srRed[roicount,:])
    statistics_srRed[roicount,1] = p

# h0, qs = mc.fdrcorrection0(statistics_srRed[:,1])
# for roicount in range(len(rois)):
#     statistics_srRed[roicount,1] = qs[roicount]
#     statistics_srRed[roicount,2] = h0[roicount]*statistics_srRed[roicount,0]
    
roi_ind = np.where(networkdef!=networkmappings['smn'])[0]
h0, qs = mc.fdrcorrection0(statistics_srRed[roi_ind,1])
statistics_srRed[:,1] = 1.0
for roicount in range(len(roi_ind)):
    roi = roi_ind[roicount]
    statistics_srRed[roi,1] = qs[roicount]
    statistics_srRed[roi,2] = h0[roicount]*statistics_srRed[roi,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_srRed[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics_srRed[sig_ind,0]

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_srRed[roicount,0]
    inputStim[vertex_ind,1] = statistics_srRed[roicount,1]
    inputStim[vertex_ind,2] = statistics_srRed[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/srRed_Regions.csv', np.where(statistics_srRed[:,1]<0.05)[0], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/'
filename = sr_rule + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)


Number of ROIs significant for srRed Stimuli: 5
Accuracies: [ 0.33854167  0.32552083  0.39583333  0.3359375   0.328125  ]


0

# 2.0 Run across subject decoding on VERTICAL SR rules
### 4-way decoding
* both vertical
* notboth vertical
* either vertical
* neither vertical

## 2.1 Load data for VERTICAL SR integration

In [99]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srVertical'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools_group.loadInputActivity(subj,sr_rule)
    scount += 1

/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


#### Run decoding 

In [100]:
nproc = 30
ncvs = 1
rois = np.arange(nParcels)

distances_baseline_srVertical = tools_group.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)


#### Compute statistics

In [101]:
sr_rule = 'srVertical'
statistics_srVertical = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_srVertical.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_srVertical[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_srVertical[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_srVertical[roicount,0] = np.mean(distances_baseline_srVertical[roicount,:])
    statistics_srVertical[roicount,1] = p

# h0, qs = mc.fdrcorrection0(statistics_srVertical[:,1])
# for roicount in range(len(rois)):
#     statistics_srVertical[roicount,1] = qs[roicount]
#     statistics_srVertical[roicount,2] = h0[roicount]*statistics_srVertical[roicount,0]
    
roi_ind = np.where(networkdef!=networkmappings['smn'])[0]
h0, qs = mc.fdrcorrection0(statistics_srVertical[roi_ind,1])
statistics_srVertical[:,1] = 1.0
for roicount in range(len(roi_ind)):
    roi = roi_ind[roicount]
    statistics_srVertical[roi,1] = qs[roicount]
    statistics_srVertical[roi,2] = h0[roicount]*statistics_srVertical[roi,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_srVertical[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics_srVertical[sig_ind,0]

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_srVertical[roicount,0]
    inputStim[vertex_ind,1] = statistics_srVertical[roicount,1]
    inputStim[vertex_ind,2] = statistics_srVertical[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/srVertical_Regions.csv', np.where(statistics_srVertical[:,1]<0.05)[0], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/'
filename = sr_rule + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)


Number of ROIs significant for srVertical Stimuli: 2
Accuracies: [ 0.328125    0.33072917]


0

# 3.0 Run across subject decoding on HIGH SR rules
### 4-way decoding
* both high
* notboth high
* either high
* neither high

## 3.1 Load data for HIGH SR integration

In [85]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srHigh'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools_group.loadInputActivity(subj,sr_rule)
    scount += 1

/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


#### Run decoding 

In [86]:
nproc = 30
ncvs = 1
rois = np.arange(nParcels)

distances_baseline_srHigh = tools_group.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)


#### Compute statistics

In [95]:
sr_rule = 'srHigh'
statistics_srHigh = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_srHigh.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_srHigh[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_srHigh[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_srHigh[roicount,0] = np.mean(distances_baseline_srHigh[roicount,:])
    statistics_srHigh[roicount,1] = p

# h0, qs = mc.fdrcorrection0(statistics_srHigh[:,1])
# for roicount in range(len(rois)):
#     statistics_srHigh[roicount,1] = qs[roicount]
#     statistics_srHigh[roicount,2] = h0[roicount]*statistics_srHigh[roicount,0]
    
roi_ind = np.where(networkdef!=networkmappings['smn'])[0]
h0, qs = mc.fdrcorrection0(statistics_srHigh[roi_ind,1])
statistics_srHigh[:,1] = 1.0
for roicount in range(len(roi_ind)):
    roi = roi_ind[roicount]
    statistics_srHigh[roi,1] = qs[roicount]
    statistics_srHigh[roi,2] = h0[roicount]*statistics_srHigh[roi,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_srHigh[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics_srHigh[sig_ind,0]

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_srHigh[roicount,0]
    inputStim[vertex_ind,1] = statistics_srHigh[roicount,1]
    inputStim[vertex_ind,2] = statistics_srHigh[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/srHigh_Regions.csv', np.where(statistics_srHigh[:,1]<0.05)[0], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/'
filename = sr_rule + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)


Number of ROIs significant for srHigh Stimuli: 5
Accuracies: [ 0.3359375   0.33072917  0.3359375   0.33333333  0.328125  ]


0

# 4.0 Run across subject decoding on CONSTANT SR rules
### 4-way decoding
* both constant
* notboth constant
* either constant
* neither constant

## 4.1 Load data for CONSTANT SR integration

In [88]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srConstant'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools_group.loadInputActivity(subj,sr_rule)
    scount += 1

/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


#### Run decoding 

In [90]:
nproc = 30
ncvs = 1
rois = np.arange(nParcels)

distances_baseline_srConstant = tools_group.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)


#### Compute statistics

In [94]:
sr_rule = 'srConstant'
statistics_srConstant = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_srConstant.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_srConstant[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_srConstant[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_srConstant[roicount,0] = np.mean(distances_baseline_srConstant[roicount,:])
    statistics_srConstant[roicount,1] = p

#h0, qs = mc.fdrcorrection0(statistics_srConstant[:,1])
#for roicount in range(len(rois)):
#    statistics_srConstant[roicount,1] = qs[roicount]
#    statistics_srConstant[roicount,2] = h0[roicount]*statistics_srConstant[roicount,0]

roi_ind = np.where(networkdef!=networkmappings['smn'])[0]
h0, qs = mc.fdrcorrection0(statistics_srConstant[roi_ind,1])
statistics_srConstant[:,1] = 1.0
for roicount in range(len(roi_ind)):
    roi = roi_ind[roicount]
    statistics_srConstant[roi,1] = qs[roicount]
    statistics_srConstant[roi,2] = h0[roicount]*statistics_srConstant[roi,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_srConstant[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics_srConstant[sig_ind,0]

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_srConstant[roicount,0]
    inputStim[vertex_ind,1] = statistics_srConstant[roicount,1]
    inputStim[vertex_ind,2] = statistics_srConstant[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/srConstant_Regions.csv', np.where(statistics_srConstant[:,1]<0.05)[0], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/SRDecoding/'
filename = sr_rule + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)


Number of ROIs significant for srConstant Stimuli: 2
Accuracies: [ 0.3359375  0.3359375]


0